In [1]:
from django_for_jupyter import init_django

init_django()

/home/javier


[nltk_data] Downloading package stopwords to /home/javier/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/javier/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [88]:
from precios.models import (
    Site, SiteURLResults, Marcas, Articulos, Vendedores, Unifica, Settings
)
from datetime import datetime
from django.db.models import  Count

In [89]:
def get_sitemap(pagina):
    suffix = 'lalala://'
    priority = '0.5'
    changefreq = "weekly"

    desde   = (( pagina - 1 ) * 100 ) + 0
    hasta   = (( pagina     ) * 100 ) - 1


    num_articles = Articulos.objects.all().count()
    num_paginas = int(num_articles / 10000) + 1
    print(f'num_articles={num_articles}  num_paginas={num_paginas}  ')

    
    articles_from_all = Vendedores.objects.select_related('articulo')
    articles_from_all = articles_from_all.exclude(vendidoen__precio=0)
    articles_from_all = articles_from_all.exclude(vendidoen__error404=True)
    articles_from_all = articles_from_all.values('articulo__pk').all()
    

    articles_count = articles_from_all.distinct().count()
    num_paginas = int(articles_count / 10000) + 1
    print(f'articles_from_all={articles_count} num_paginas={num_paginas}')
    
    articles_from_all2 = articles_from_all
    records = Articulos.objects.filter(id__in=articles_from_all2)[desde:hasta]
    urlset = []
    for rec in records:
        row = {'location': suffix + rec.get_absolute_url(), 'lastmod': rec.created.strftime('%Y-%m-%d'), 'priority': priority, 'changefreq': changefreq}
        urlset.append(row)
    
    # print(art.get_absolute_url())

    print(urlset[1])


In [104]:
def sitemaps_marcas( pagina):
    pagina              = int(pagina)
    num_records         = 10000
    suffix              = 'lalala://'
    desde               = (( pagina - 1 ) * num_records ) + 0
    hasta               = (( pagina     ) * num_records ) - 1
    priority            = '0.5'
    changefreq          = "weekly"
    MinSuperCompara     = int(Settings.objects.get(key='MinSuperCompara').value)
    lastmod             = datetime.today().strftime('%Y-%m-%d')

    articles_from_all = Vendedores.objects.select_related('articulo')
    articles_from_all = articles_from_all.exclude(vendidoen__precio=0)
    articles_from_all = articles_from_all.exclude(vendidoen__error404=True)
    articles_from_all = articles_from_all.exclude(articulo__marca__es_marca=False)
    articles_from_all = articles_from_all.values('articulo__marca__pk').distinct().all()
    print(f'posibles_marcas={articles_from_all.count()}')

    
    records     = Marcas.objects.filter(id__in=articles_from_all).filter(es_marca=True)
    num_paginas = int(Marcas.objects.filter(id__in=articles_from_all).count() / num_records) + 1

    # print(f'actuales_marcas={Marcas.objects.filter(id__in=subquery).count()}')
        
    records =  records[desde:hasta]

    urlset = []
    for rec in records:
        urlset.append({'location': suffix + rec.get_absolute_url(), 'lastmod': lastmod, 'priority': priority, 'changefreq': changefreq})

    print(urlset[1])

In [105]:
get_sitemap(3)
sitemaps_marcas(1)

num_articles=59088  num_paginas=6  
articles_from_all=57910 num_paginas=6
{'location': 'lalala:///precios/article/4m-malla-geotextil-t-x-mt-lineal-100-mtlxrollo-1/', 'lastmod': '2023-10-29', 'priority': '0.5', 'changefreq': 'weekly'}
posibles_marcas=3045
{'location': 'lalala:///precios/precios/?marca=1115541', 'lastmod': '2023-10-30', 'priority': '0.5', 'changefreq': 'weekly'}
